<a href="https://www.kaggle.com/code/tunguz/mnist-xgboost-baseline?scriptVersionId=96703806" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.preprocessing import RobustScaler
import gc

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')
sample_submission = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
col = ['pixel%d'%i for i in range(784)]

In [3]:
target = train[['label']].values
train.drop(['label'], axis=1, inplace=True)

In [4]:
features = train.columns

In [5]:
%%time
train_test = pd.concat([train, test], axis =0)
RS = RobustScaler()
RS.fit(train_test)
del train_test
gc.collect()
gc.collect()
train[features] = RS.transform(train)
test[features] = RS.transform(test)
del RS
gc.collect()
gc.collect()

CPU times: user 43 s, sys: 1min 25s, total: 2min 8s
Wall time: 2min 9s


0

In [6]:
X_train, X_val, Y_train, Y_val = train_test_split(train, target, test_size=0.1, random_state=42)
dtrain = xgb.DMatrix(X_train, label=Y_train)
dval = xgb.DMatrix(X_val, label=Y_val)

In [7]:
%%time

watchlist = [(dval, 'eval'), (dtrain, 'train')]

xgb_params = {'objective': 'multi:softprob',
 'tree_method': 'gpu_hist',
 'lambda': 0.0014345301923969806,
 'alpha': 0.4687380486697353,
 'colsample_bytree': 0.8434345456331338,
 'subsample': 0.8991048362147761,
 'learning_rate': 0.02,
 'max_depth': 21,
 'min_child_weight': 4,
 'num_class': 10,
 'eval_metric': 'merror'}

xgb_clf = xgb.train(xgb_params, dtrain, 5000, 
                    watchlist, 
                    #early_stopping_rounds=40, 
                    verbose_eval=100)

[0]	eval-merror:0.109762	train-merror:0.060847
[100]	eval-merror:0.039048	train-merror:0.003571
[200]	eval-merror:0.03619	train-merror:0.000794
[300]	eval-merror:0.033095	train-merror:0.000185
[400]	eval-merror:0.030952	train-merror:5.3e-05
[500]	eval-merror:0.030238	train-merror:2.6e-05
[600]	eval-merror:0.028333	train-merror:0
[700]	eval-merror:0.027619	train-merror:0
[800]	eval-merror:0.027381	train-merror:0
[900]	eval-merror:0.027143	train-merror:0
[1000]	eval-merror:0.026905	train-merror:0
[1100]	eval-merror:0.026667	train-merror:0
[1200]	eval-merror:0.026667	train-merror:0
[1300]	eval-merror:0.02619	train-merror:0
[1400]	eval-merror:0.02619	train-merror:0
[1500]	eval-merror:0.02619	train-merror:0
[1600]	eval-merror:0.025952	train-merror:0
[1700]	eval-merror:0.025714	train-merror:0
[1800]	eval-merror:0.025476	train-merror:0
[1900]	eval-merror:0.025476	train-merror:0
[2000]	eval-merror:0.025476	train-merror:0
[2100]	eval-merror:0.025238	train-merror:0
[2200]	eval-merror:0.025238	tr

In [8]:
res_prob = xgb_clf.predict(xgb.DMatrix(test[col]))


In [9]:
res_prob

array([[7.7722198e-06, 1.8453152e-07, 9.9998176e-01, ..., 1.5053104e-06,
        2.3649222e-06, 1.4763203e-06],
       [9.9996936e-01, 1.4947316e-06, 8.6879609e-06, ..., 2.1686126e-06,
        8.8594805e-07, 3.5794204e-07],
       [1.4276125e-05, 3.6797144e-05, 3.7631323e-04, ..., 2.0257063e-04,
        5.9644697e-04, 9.9730599e-01],
       ...,
       [2.0978821e-07, 5.7093388e-07, 3.4353511e-07, ..., 2.1717375e-07,
        2.3512862e-06, 1.6858443e-06],
       [2.3115397e-05, 1.1602393e-06, 5.3967813e-07, ..., 1.0445291e-05,
        2.8931458e-06, 9.9951577e-01],
       [2.0246034e-06, 3.2667805e-07, 9.9996305e-01, ..., 1.2065478e-06,
        6.2818690e-06, 1.3418520e-05]], dtype=float32)

In [10]:
np.argmax(res_prob, axis=1)

array([2, 0, 9, ..., 3, 9, 2])

In [11]:
val_preds = np.argmax(xgb_clf.predict(dval), axis=1)

In [12]:
val_preds

array([8, 1, 9, ..., 7, 7, 5])

In [13]:
accuracy_score(Y_val, val_preds)

0.9757142857142858

In [14]:
res = np.argmax(res_prob, axis=1)

In [15]:
res.astype('int')

array([2, 0, 9, ..., 3, 9, 2])

In [16]:
sample_submission['Label'] = res.astype('int')
sample_submission.to_csv('submission.csv', index=False)
np.save('res_prob', res_prob)